In [ ]:
""" Quel périmetre?

- visite du site:
    - trustpilot filtre sur le pays (truspilot US, truspilot FR etc.)
    - catégories -> 22 (aliment, élétronique, santé etc.)
    - sous-catégories -> une dizaine max par catégories
        - pages catégorie -> nombre d'avis, score, le lieu, nom entreprise, sous-cat (+)
        - pages catégorie -> trier par avis pertinent, avec le plus d'avis, avis les plus recent
            - pages entreprise -> (+) titre de l'avis, le texte de l'avis, le lieu de l'avis, la date, le username, nombre davis effectué par la personne, avis verifié ou pas
            - pages entreprise -> réponse de l'entreprise, date
                - pages personne -> tous ces avis toute entreprise confondue

- on peut commencer par prendre le site Truspilote en francais -> les entreprises en france
- toute la france
- quelles catégories?
    - 
- problématique: 
    - application du point de vue du marchant, << à propos de la satisfaction client ou je me situe par rapport à mes concurrents? >>
    - << qu'es ce qui marche chez mes concurrents et qui ne marche pas chez moi ? >>
        - proportion d'avis négatifs
        - proportion de ceux qui repondent aux avis negatifs
        - afficher le top 3 des avis positifs et négatifs
        - calculer un score
        - comparer les termes positifs et negatifs entre l'entreprise et ses concurrents

- liens:
    - https://www.conseilsmarketing.com/fidelisation/comment-calculer-un-nps-et-quest-ce-quun-bon-score/

"""

In [ ]:
"""
Quelle solution pour récuperer de la données?:
- Scrapping
- API truspilot doc ici https://documentation-apidocumentation.trustpilot.com/
    - nécessite un compte business (demande validation que le site web exite vraiment)
"""

In [2]:
# libs

from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import pandas as pd

In [3]:
# import categories et sub_categories

tpPage = urlopen("https://fr.trustpilot.com/categories")
soup = BeautifulSoup(tpPage, 'html.parser')

tpCategories = soup.findAll(name = 'a', href = True, attrs = {'class': 'styles_headingLink__3ESdh'})
tpSubCat = soup.findAll(name = 'a', href = True, attrs = {'class': 'link_notUnderlined__1vRfl'})


cat = []
for element in tpCategories:
        cat.append(element["href"])

subCat = []
for element in tpSubCat:
        subCat.append(element["href"])

df_cat = pd.DataFrame(list(zip(cat)), columns = ["Category"])
df_subcat = pd.DataFrame(list(zip(subCat)), columns = ["Sub_category"])
# todo faire un seul dataframe regroupant cat et sous_cat

In [4]:
df_cat.head()

,Category
0,/categories/food_beverages_tobacco
1,/categories/animals_pets
2,/categories/money_insurance
3,/categories/beauty_wellbeing
4,/categories/construction_manufactoring


In [5]:
# display merchant details


# https://fr.trustpilot.com/categories/electronics_technology


def displayScrapData(n, url_sub_category = "https://fr.trustpilot.com/categories/electronics_technology"):

    """
    def: function to print the n first lines from the Trustpilote website
    param:
        n: number of line to display
        url_sub_category: url of the sub_category from the trustpilote website
    """
    
    tpPage = urlopen(url_sub_category)
    soup = BeautifulSoup(tpPage, 'html.parser')

    tpTitles = soup.findAll(name = 'p', attrs = {'class': 'styles_displayName__1LIcI'})
    tpMetrics = soup.findAll(name = 'p', attrs = {'class': 'styles_ratingText__nheL7'})
    tpPlaces = soup.findAll(name = 'p', attrs = {'class': 'styles_location__3JATO'})
    tpSubCategories = soup.findAll(name = 'div', attrs = {'class': 'styles_desktop__3N0-b'})
    tpWebsite = soup.findAll(name = 'a', href = True, attrs = {'class': 'styles_linkWrapper__3x9X7'})


    title = [] # On crée une liste vide qui contiendra tous les titres propres
    for element in tpTitles:
        title.append(element.text.encode('ascii', errors='ignore').decode('utf-8'))

    rating = []
    review = []
    for element in tpMetrics:
        rating.append(element.text.encode('ascii', errors='ignore').decode('utf-8').split()[1].split("|")[0])
        review.append(element.text.encode('ascii', errors='ignore').decode('utf-8').split()[1].split("|")[1])

    city = []
    country = []
    for element in tpPlaces:
        city.append(element.text.encode('ascii', errors='ignore').decode('utf-8').split(",")[0])
        country.append(element.text.encode('ascii', errors='ignore').decode('utf-8').split(",")[1])

    subCategory = []
    for element in tpSubCategories:
        subCategory.append(element.text.encode('ascii', errors='ignore').decode('utf-8'))

    website = []
    for element in tpWebsite:
        website.append("https://fr.trustpilot.com" + element["href"])

    df = pd.DataFrame(list(zip(title, rating, review, website)), columns = ["Title", "Rating", "Nb_of_reviews", "Website"])

    return df.head(n)

In [6]:
displayScrapData(20)

,Title,Rating,Nb_of_reviews,Website
0,Cleanfox,"4,9",7197,https://fr.trustpilot.com/review/www.cleanfox.io
1,Fonebank.fr,"4,9",4157,https://fr.trustpilot.com/review/fonebank.fr
2,Affinitweet,"4,9",4078,https://fr.trustpilot.com/review/affinitweet.com
3,Spliiit,"4,9",2209,https://fr.trustpilot.com/review/spliiit.com
4,WP Rocket,"4,9",1623,https://fr.trustpilot.com/review/wp-rocket.me
5,TransferNow,"4,9",1509,https://fr.trustpilot.com/review/transfernow.net
6,Europe Nature-optik,"4,9",1315,https://fr.trustpilot.com/review/www.europe-na...
7,sonner.tel,"4,9",544,https://fr.trustpilot.com/review/sonner.tel
8,Greenvallee,"4,9",451,https://fr.trustpilot.com/review/greenvallee.fr
9,Docorga,"4,9",447,https://fr.trustpilot.com/review/www.docorga.com


In [7]:
#display notes details

# todo à revoir

tpPage = urlopen("https://fr.trustpilot.com/review/www.cleanfox.io")
soup = BeautifulSoup(tpPage, 'html.parser')

tpNicknames = soup.findAll(name = 'h2', attrs = {'class': 'typography_appearance-default__AAY17'})
tpComments = soup.findAll(name = 'p', attrs = {'class': 'typography_color-black__5LYEn'})


titleComments = [] # 
for element in tpNicknames:
    titleComments.append(element.text.encode('ascii', errors='ignore').decode('utf-8'))

comment = [] # 
for element in tpComments:
    comment.append(element.text.encode('ascii', errors='ignore').decode('utf-8'))

df_comm = pd.DataFrame(list(zip(comment)), columns = ["Comments"])

df_comm.head()


,Comments
0,J'adore cette appli qui permet de dsencombrer ...
1,Date de l'exprience: 12 octobre 2022
2,"J'utilise Cleanfox rgulirement, c'est un suer ..."
3,Date de l'exprience: 12 octobre 2022
4,BonjourAu dbut jtais un peu pessimiste car je ...
